В качестве финального проекта реализована система подсказок похожих вопросов на данных сайта Quora. Поиск производится исключительно по основному заголовку без уточняющих деталей.

Система представлена микросервисом на основе Flask. Верхнеуровнево пайплайн и критерии можно представить так:

Сначала происходит фильтрация запроса по языку (с помощью библиотеки LangDetect) — исключаются все запросы, для которых определённый язык не равняется "en". Затем происходит поиск вопросов-кандидатов с помощью FAISS (по схожести векторов) — в этой части предлагается ограничиться векторизацией только тех слов, эмбеддинги которых есть в исходных GLOVE-векторах. Эти кандидаты реранжируются KNRM-моделью, после чего до 10 кандидатов выдаются в качестве ответа.


На сервере реализовано две ручки: для запросов (для поиска похожих вопросов) и для создания FAISS-индекса.

/query — принимает POST-запрос. Должна вернуть json, где status='FAISS is not initialized!' в случае, если в решение не были загружены вопросы для поиска с помощью второго метода. 

Формат запроса для query:

json-запрос, с единственным ключом 'queries', значение которого — список строк с вопросами (Dict[str, List[str]]).

Формат ответа (в случае созданного индекса) — json с двумя полями. lang_check описывает, был ли распознан запрос как английский (List[bool], True/False-значения), suggestions — List[Optional[List[Tuple[str, str]]]].

В этом списке для каждого запроса из query необходимо указать список (до 10) найденных схожих вопросов, где каждый вопрос представлен в виде Tuple, в котором первое значение — id текста (см. ниже), второе — сам непредобработанный текст схожего вопроса. Если проверка на язык не пройдена (не английский), либо произошёл какой-то сбой в обработке — оставьте None в списке вместо ответа (например, [[(..., ...), (..., ...), ...], None, ... ]).

/update_index — принимает POST-запрос, в котором в json присутствует поле documents, Dict[str,str] — все документы, где ключ — id текста, значение — сам текст. На предобработку и создание индекса даётся 200 секунд. Подразумевается, что инициализация происходит единоразово, поэтому не нужно беспокоиться о повторном вызове этого метода. В возвращаемом json'е должно быть два ключа: status (ok, если всё прошло гладко) и index_size, значение которого — единственное целое число, хранящее количество документов в индексе.

В ноутбуке реализована демонстрация работы сервиса. 

In [1]:
from IPython.display import display, HTML
display(HTML("<style>:root { --jp-notebook-max-width: 95% !important; }</style>"))

In [2]:
user_query = input("Quora question (Is it possible to breathe through one's butt? as example):")

Quora question (Is it possible to breathe through one's butt? as example): Country for relaxation in September.


In [3]:
import pandas as pd, numpy as np
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 255)

In [4]:
import requests, json
from typing import Dict, List, Tuple, Union, Callable, Optional
import builder 
# URL Flask app
base_url = 'http://127.0.0.1:5000'

class Solution:
  # import QUORA data + debug functions 
  # generate query, generate data for initial  FAISS indexing
  def __init__(
      self,
      glue_qqp_dir,
      ):
    self.glue_qqp_dir = glue_qqp_dir
    self.glue_df_train = self.get_glue_df('train')
    self.glue_df_test = self.get_glue_df('dev')
    self.json_query = self._generate_json_server_query()
    self.json_documents = self._generate_json_server_documents()

  def get_glue_df(self, partition_type: str) -> pd.DataFrame:
      assert partition_type in ['dev', 'train']
      if pd.__version__ > '1.3.0':
        glue_df = pd.read_csv(
            self.glue_qqp_dir + f'/{partition_type}.tsv', sep='\t', on_bad_lines='skip', dtype=object)
      else:
        glue_df = pd.read_csv(
            self.glue_qqp_dir + f'/{partition_type}.tsv', sep='\t', error_bad_lines=False, dtype=object)
      glue_df = glue_df.dropna(axis=0, how='any').reset_index(drop=True)
      glue_df_fin = pd.DataFrame({
          'id_left': glue_df['qid1'],
          'id_right': glue_df['qid2'],
          'text_left': glue_df['question1'],
          'text_right': glue_df['question2'],
          'label': glue_df['is_duplicate'].astype(int)
      })
      return glue_df_fin

  def _generate_json_server_query(self) -> Dict[str, List[str]]:
    # generating query for debug
    return {'query' : self.glue_df_test['text_left'].sample(10).tolist() + ['Этот запрос на другом языке', user_query]}

  def _generate_json_server_documents(self) -> Dict[str, Dict[str, List[str]]]:
    # generating documents for debug
    left_dict = self.glue_df_train[['id_left', 'text_left']].set_index('id_left').to_dict()['text_left']
    # right_dict = self.glue_df_train[['id_right', 'text_right']].set_index('id_right').to_dict()
    # documents = left_dict.update(right_dict)
    return {'documents' : left_dict}   
  

def query() -> Tuple:
    # return random queries from QUORA test.tsv dataset
    url = f'{base_url}/query'
    payload = sol._generate_json_server_query()

    response = requests.post(url, data=json.dumps(payload), headers=headers)
    return response.json(), payload

def update_index() -> Dict:
   # return QUORA train.tsv text documents to init FAISS index
   url = f'{base_url}/update_index'

   payload = sol._generate_json_server_documents()
   response = requests.post(url, data=json.dumps(payload), headers=headers)
   return response.json()

if __name__ == '__main__':
    # init QUORA dataset
    sol = Solution(builder.glue_qqp_dir)
    headers = {'Content-Type': 'application/json'}
    
    # POST query on server, check without FAISS index update
    query_response, _ = query()
    print('Query Response: ',  query_response)
    
    # POST documents on server to init FAISS indexes
    update_response = update_index()
    print('Update Response: ', update_response)
    
    # POST query on server
    query_response, query = query()
    print('Query Response: ',  query_response)

Query Response:  {'status': 'FAISS is not initialized!'}
Update Response:  {'status': 'ok'}
Query Response:  {'lang_check': [True, True, True, True, True, True, True, True, True, True, False, True], 'suggestions': [[['85277', 'What is the purpose of dreaming?'], ['376395', 'What is the purpose of AirWatch?'], ['420663', 'What is the purpose of life?'], ['40354', 'What is the purpose of life?'], ['44023', 'What is the purpose of life ?'], ['240159', 'What is the purpose of education?'], ['329784', 'What is the origin of morality?'], ['304996', 'What is the aim of religions?'], ['191890', 'What is the meaning of humanity?'], ['262871', 'What is the philosophy of Existence?']], [['513413', 'Is demonetisation good or bad for India?'], ['145436', 'What is good and bad about squarespace?'], ['106103', 'What is a bad form of love?'], ['494336', 'Which one is better, Kopaja or Metromini? Why?'], ['435132', 'Is it good to be sad sometimes?'], ['22884', 'Is it bad to like someone?'], ['198807', 

In [5]:
# Display query and ranked by KNRM suggestion from server 
suggestion_arr = np.array(query_response['suggestions'])[:,:,1]
N = suggestion_arr.shape[1] # count ranked documents for each query
query_arr =  np.array(query['query']).reshape(-1,1)

query_df = pd.DataFrame(np.repeat(query_arr, N), columns = ['query'])
suggestion_df = pd.DataFrame(suggestion_arr.reshape(-1), columns = ['suggestions'])

pd.concat([query_df, suggestion_df], axis = 1).set_index(['query', 'suggestions'])

Empty DataFrame
Columns: []
Index: [(What is the purpose of religion?, What is the purpose of dreaming?), (What is the purpose of religion?, What is the purpose of AirWatch?), (What is the purpose of religion?, What is the purpose of life?), (What is the purpose of religion?, What is the purpose of life?), (What is the purpose of religion?, What is the purpose of life ?), (What is the purpose of religion?, What is the purpose of education?), (What is the purpose of religion?, What is the origin of morality?), (What is the purpose of religion?, What is the aim of religions?), (What is the purpose of religion?, What is the meaning of humanity?), (What is the purpose of religion?, What is the philosophy of Existence?), (Is having lots of crushes bad or good?, Is demonetisation good or bad for India?), (Is having lots of crushes bad or good?, What is good and bad about squarespace?), (Is having lots of crushes bad or good?, What is a bad form of love?), (Is having lots of crushes bad or good?, Which one is better, Kopaja or Metromini? Why?), (Is having lots of crushes bad or good?, Is it good to be sad sometimes?), (Is having lots of crushes bad or good?, Is it bad to like someone?), (Is having lots of crushes bad or good?, Is being "different" a bad thing?), (Is having lots of crushes bad or good?, Is too much pineapple bad for you?), (Is having lots of crushes bad or good?, Is being shy and reserved a bad thing?), (Is having lots of crushes bad or good?, What is more precious, love or money?), (How can you tell if a guy likes you or not?, How can you tell if a guy likes you or not?), (How can you tell if a guy likes you or not?, How do you know if you're a sociopath or not?), (How can you tell if a guy likes you or not?, How do you know if you like a book or not?), (How can you tell if a guy likes you or not?, How can you tell if you're really an introvert or an ambivert?), (How can you tell if a guy likes you or not?, How do I know if a guy likes me or not?), (How can you tell if a guy likes you or not?, How do you know if a guy is playing you?), (How can you tell if a guy likes you or not?, How do you know if a guy is obsessed with you?), (How can you tell if a guy likes you or not?, How do you know if you are a good parent?), (How can you tell if a guy likes you or not?, How do you know if a person loves you back?), (How can you tell if a guy likes you or not?, How do you know your boyfriend is not inlove with you?), (How banning 500 and 1000 rs note will help fight corruption in India?, How will banning Rs 500 and Rs 1000 note help fight corruption in India?), (How banning 500 and 1000 rs note will help fight corruption in India?, How banning 500 and 1000 rs note will help fight corruption in India?), (How banning 500 and 1000 rs note will help fight corruption in India?, How will banning Rs. 500 and Rs. 1000 notes help in overcoming corruption in India?), (How banning 500 and 1000 rs note will help fight corruption in India?, So how does banning 500, 1000 rs and introducing 2000 rs will curb corruption?), (How banning 500 and 1000 rs note will help fight corruption in India?, How will demonetising ₹500 and ₹1,000 notes help reduce corruption in India?), (How banning 500 and 1000 rs note will help fight corruption in India?, How will the banning of Rs. 500 and Rs. 1000 notes affect the economy?), (How banning 500 and 1000 rs note will help fight corruption in India?, How does scrapping the Rs 1000 and 500 notes help in curbing terrorism?), (How banning 500 and 1000 rs note will help fight corruption in India?, How is banning 500 and 1000 INR going to help Indian economy?), (How banning 500 and 1000 rs note will help fight corruption in India?, How the Indian economy will get affected after banning Rs 500 and 1000 notes?), (How banning 500 and 1000 rs note will help fight corruption in India?, How will the ban of Rs 500 and Rs 1000 notes affect Pakistan?), (What does the Hindi word "Baregama" mean?, What does the Hindi word "